In [1]:
#import pandas as pd 
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))



2025-09-20 17:00:56.303032: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-20 17:00:57.805946: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI AVX_VNNI_INT8 AVX_NE_CONVERT FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-20 17:01:23.507943: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


W0000 00:00:1758412897.929369  154313 gpu_device.cc:2431] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 12.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.


In [2]:
import numpy as np 

In [3]:
print("Hello ese")

Hello ese


In [4]:
!python --version
print('Numpy ' + np.__version__)
print('TensorFlow ' + tf.__version__)

Python 3.12.3
Numpy 2.2.6
TensorFlow 2.20.0


In [5]:

import torch 
import torch.nn as nn 
print(torch.__version__)

2.8.0+cu128


In [6]:


class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, stride, bias=False, padding_mode="reflect"), 
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2), 
        )

    def forward(self, x):
        return self.conv(x)
    
class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]): 
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels*2, features[0], kernel_size=4, stride=2, padding=1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),  
        )

        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(
                CNNBlock(in_channels, feature, stride=1 if feature == features[-1] else 2),
            ) 
            in_channels = feature

        
        layers.append(
            nn.Conv2d(
                in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect"
            ),
        )


        self.model = nn.Sequential(*layers)

    def forward(self, x, y): 
        x = torch.cat([x, y], dim=1)
        x = self.initial(x)
        return self.model(x)


In [7]:
def test():
    x = torch.randn((1, 3, 256, 256))
    y = torch.randn((1, 3, 256, 256))
    model = Discriminator()
    preds = model(x, y)
    print(preds.shape)



In [8]:
if __name__ == "__main__":
    test()

torch.Size([1, 1, 26, 26])


In [9]:
###################################

In [10]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, act="relu", use_dropout=False):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False, padding_mode="reflect")
            if down 
            else nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias = False), 
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act == "relu" else nn.LeakyReLU(0.2), 

        )
        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.conv(x)
        return self.dropout(x) if self.use_dropout else x 
    

class Generator(nn.Module):
    def __init__(self, in_channels = 3, features=64): 
        super().__init__()   
        self.initial_down = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),

        ) #128 after this one 

        self.down1 = Block(features, features*2, down=True, act="leaky", use_dropout=False )    #64x64 
        self.down2 = Block(features*2, features*4, down=True, act="leaky", use_dropout=False )  #32x32 
        self.down3 = Block(features*4, features*8, down=True, act="leaky", use_dropout=False )  #16
        self.down4 = Block(features*8, features*8, down=True, act="leaky", use_dropout=False )  #8
        self.down5 = Block(features*8, features*8, down=True, act="leaky", use_dropout=False )  #4 
        self.down6 = Block(features*8, features*8, down=True, act="leaky", use_dropout=False )  #2
        self.bottleneck = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1, padding_mode="reflect"), nn.ReLU(), #1x1
        )
        self.up1 = Block(features*8, features*8, down=False, act="relu", use_dropout=True) #2x2  
        self.up2 = Block(features*8*2, features*8, down=False, act="relu", use_dropout=True) #4x4
        self.up3 = Block(features*8*2, features*8, down=False, act="relu", use_dropout=True) #8x8
        self.up4 = Block(features*8*2, features*8, down=False, act="relu", use_dropout=False) #16x16
        self.up5 = Block(features*8*2, features*4, down=False, act="relu", use_dropout=False) #32x32
        self.up6 = Block(features*4*2, features*2, down=False, act="relu", use_dropout=False) #64x64
        self.up7 = Block(features*2*2, features, down=False, act="relu", use_dropout=False) #128x128
        self.final_up = nn.Sequential(
            nn.ConvTranspose2d(features*2, in_channels, kernel_size=4, stride=2, padding=1), 
            nn.Tanh(),  #256x256
        )

    def forward(self, x): 
        d1 = self.initial_down(x) #128x128 
        d2 = self.down1(d1)     #64x64  
        d3 = self.down2(d2)     #32x32 
        d4 = self.down3(d3)     #16x16
        d5 = self.down4(d4)     #8x8
        d6 = self.down5(d5)     #4x4
        d7 = self.down6(d6)     #2x2
        bottleneck = self.bottleneck(d7) #1x1
        up1 = self.up1(bottleneck)      #2x2
        up2 = self.up2(torch.cat([up1, d7], 1)) #4x4
        up3 = self.up3(torch.cat([up2, d6], 1)) #8x8
        up4 = self.up4(torch.cat([up3, d5], 1)) #16x16
        up5 = self.up5(torch.cat([up4, d4], 1)) #32x32
        up6 = self.up6(torch.cat([up5, d3], 1)) #64x64
        up7 = self.up7(torch.cat([up6, d2], 1)) #128x128
        
        return self.final_up(torch.cat([up7, d1 ], 1)) #256x256 
        




      


In [11]:
def test1(): 
    x = torch.randn((1, 3, 256, 256))
    model = Generator(in_channels=3, features=64)
    preds= model(x)
    print(preds.shape)




In [12]:
if __name__ == "__main__":
    test1()

torch.Size([1, 3, 256, 256])


In [13]:
################################################################ 
#####Config.py 



##############################################################

In [14]:

#######################################################################################################################################
#Implement DATAset 


In [15]:
from PIL import Image 
import numpy as np 
import os 
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image 
import sys 

import config 


cuda


In [ ]:
class MapDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.list_files = os.listdir(self.root_dir)
        #print(self.list_files)

    def __len__(self):
        return len(self.list_files)
    
    def __getitem__(self, index):
        img_file = self.list_files[index]
        img_path = os.path.join(self.root_dir, img_file)
        image = np.array(Image.open(img_path))
        input_image = image[:, :600, :] #images are 600x600 (so 1200 in width )
        target_image = image[:, 600:, :]

        augmentations = config.both_transform(image=input_image, image0=target_image)
        input_image = augmentations["image"]
        target_image = augmentations["image0"]

        input_image = config.transform_only_input(image=input_image)["image"]
        target_image = config.transform_only_mask(image=target_image)["image"]

        return input_image, target_image
    
if __name__  == "__main__":
    dataset = MapDataset("data/maps/train/")
    loader = DataLoader(dataset, batch_size = 5)
    for x, y in loader: 
        print(x.shape)
        save_image(x, "x.png")
        save_image(y, "y.png")
        import sympy
        
        sys.exit()
        

['1.jpg', '10.jpg', '100.jpg', '1000.jpg', '1001.jpg', '1002.jpg', '1003.jpg', '1004.jpg', '1005.jpg', '1006.jpg', '1007.jpg', '1008.jpg', '1009.jpg', '101.jpg', '1010.jpg', '1011.jpg', '1012.jpg', '1013.jpg', '1014.jpg', '1015.jpg', '1016.jpg', '1017.jpg', '1018.jpg', '1019.jpg', '102.jpg', '1020.jpg', '1021.jpg', '1022.jpg', '1023.jpg', '1024.jpg', '1025.jpg', '1026.jpg', '1027.jpg', '1028.jpg', '1029.jpg', '103.jpg', '1030.jpg', '1031.jpg', '1032.jpg', '1033.jpg', '1034.jpg', '1035.jpg', '1036.jpg', '1037.jpg', '1038.jpg', '1039.jpg', '104.jpg', '1040.jpg', '1041.jpg', '1042.jpg', '1043.jpg', '1044.jpg', '1045.jpg', '1046.jpg', '1047.jpg', '1048.jpg', '1049.jpg', '105.jpg', '1050.jpg', '1051.jpg', '1052.jpg', '1053.jpg', '1054.jpg', '1055.jpg', '1056.jpg', '1057.jpg', '1058.jpg', '1059.jpg', '106.jpg', '1060.jpg', '1061.jpg', '1062.jpg', '1063.jpg', '1064.jpg', '1065.jpg', '1066.jpg', '1067.jpg', '1068.jpg', '1069.jpg', '107.jpg', '1070.jpg', '1071.jpg', '1072.jpg', '1073.jpg', '107

SystemExit: 

/mnt/c/Users/edgar/ml/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#####train.py  

#import torch 
from utils import save_checkpoint, load_checkpoint, save_some_examples
import torch.nn as nn 
import torch.optim as optim 
import config 
#from dataset import MapDataset
#from generator_model import Generator
#from discriminator_model import Discriminator 
from torch.utils.data import DataLoader 
from tqdm import tqdm
from torchvision.utils import save_image 

def train_fn(disc, gen, opt_disc, opt_gen, l1, bce, g_scalar, d_scaler):
    loop = tqdm(loader, leave=True)

    for idx, (x, y) in enumerate(loop):
        x, y = x.to(config.DEVICE), y.to(config.DEVICE)

        #train the Discriminator
        with torch.cuda.amp.autocast(): 
            y_fake = gen(x)
            D_real = disc(x, y)
            D_fake = disc(x, y_fake.detach())
            D_real_loss = bce(D_real, torch.ones_like(D_real))
            D_fake_loss = bce(D_fake, torch.zeros_like(D_fake))
            D_loss = (D_real_loss + D_fake_loss) / 2
        
        disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        #Train Generator 
        with torch.cuda.amp.autocast():
            D_fake = disc(x, y_fake)
            G_fake_loss = bce(D_fake, torch.ones_like(D_fake))
            L1 = l1(y_fake, y)*config.L1_LAMBDA
            G_loss = G_fake_loss + L1 

        opt_gen.zero_grad()
        g_scalar.scale(G_loss).backward()
        g_scalar.step(opt_gen)
        g_scalar.update()


    

def main():
    disc = Discriminator(in_channels=3).to(config.DEVICE)
    gen = Generator(in_channels=3).to(config.DEVICE)
    opt_disc = optim.Adam(disc.parameters(), lr=config.LEARNING_RATE, betas=(0.5, 0.999))
    opt_gen = optim.Adam(gen.parameters(), lr=config.LEARNING_RATE, betas=(0.5, 0.999))
    BCE = nn.BCEWithLogitsLoss()
    L1_LOSS = nn.L1Loss()

    if config.LOAD_MODEL: 
        load_checkpoint(config.CHECKPOINT_GEN, gen, opt_gen, config.LEARNING_RATE)
        load_checkpoint(config.CHECKPOINT_DISC, disc, opt_disc, config.LEARNING_RATE)

    train_dataset = MapDataset(root_dir="data/maps/train")
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=config.NUM_WORKERS)
    g_scalar = torch.cuda.amp.GradScaler()
    d_scalar = torch.cuda.amp.GradScaler()
    val_dataset = MapDataset(root_dir="data/maps/val")
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

    for epoch in range(config.NUM_EPOCHS):
        train_fn(disc, gen, train_loader, opt_gen, L1_LOSS, BCE, g_scalar, d_scalar)

        if config.SAVE_MODEL and epoch % 5 == 0:
            save_checkpoint(gen, opt_gen, filename=config.CHECKPOINT_GEN)
            save_checkpoint(disc, opt_disc, filename=config.CHECKPOINT_DISC)

        
        save_some_examples(gen, val_loader, epoch, folder="evaluation")



if __name__ ==  "__main__":
    main()


: 